In [1]:
import ee
import sys
import time
import numpy as np
import pandas as pd
import geopandas as gpd
from tqdm import tqdm
from eeSAR.s1 import s1_collection, s1_timescan
from datetime import datetime as dt, timedelta
import ee
ee.Initialize()

In [2]:
def addAux(image):

    def set_resample(image):
        """ Set resampling of the image to bilinear"""
        return image.resample()
    
    def addGLDAS(image):

        def add_date_difference(image):

            return image.set(
                'dateDist',
                ee.Number(image.get('system:time_start')).subtract(t.millis()).abs()
            )
    
        t = image.date()
        fro = t.advance(ee.Number(-30), 'days')
        #to = t.advance(ee.Number(10), 'days')

        gldas = ee.ImageCollection("NASA/GLDAS/V021/NOAH/G025/T3H") \
            .select('SoilMoi0_10cm_inst') \
            .filterBounds(image.geometry()) \
            .filterDate(fro, t) \
            .map(set_resample) \
            .map(add_date_difference)
            
        #gldas_stat = gldas.reduce(
        #        ee.Reducer.mean().combine(ee.Reducer.stdDev(), None, True)
        #    ).rename('gldas_mean', 'gldas_stddev')
        
        #gldas = gldas.filterDate(fro, t).map(add_date_difference)
            
        sm_gldas = gldas.sort('dateDist').first().rename('sm_1')

        gldas_3day = gldas.filterDate(t.advance(ee.Number(-3), 'days'), t)
        gldas_3day = gldas_3day.sum().divide(gldas_3day.count()).rename('sm_3')

        gldas_7day = gldas.filterDate(t.advance(ee.Number(-7), 'days'), t)
        gldas_7day = gldas_7day.sum().divide(gldas_7day.count()).rename('sm_7')

        gldas_30day = gldas.sum().divide(gldas.count()).rename('sm_30')

        # image = image.addBands(gldas_stat)
        return image.addBands(sm_gldas).addBands(gldas_3day).addBands(gldas_7day).addBands(gldas_30day)
    
    
    def addGPM(image):

        def add_date_difference(image):

            return image.set(
                'dateDist',
                ee.Number(image.get('system:time_start')).subtract(t.millis()).abs()
            )

        t = image.date()
        # t = ee.Date(feature.get('date').getInfo()['value'])
        fro = t.advance(ee.Number(-30), 'days')

        gpm = ee.ImageCollection('NASA/GPM_L3/IMERG_V06') \
            .filterBounds(image.geometry()) \
            .filterDate(fro, t) \
            .select('HQprecipitation') \
            .map(add_date_difference) \
            .map(set_resample)


        gpm_closest = gpm.filterDate(t.advance(ee.Number(-1), 'days'), t)  
        gpm_closest = gpm_closest.sum().divide(gpm_closest.count()).rename('precipitat')

        gpm_3day = gpm.filterDate(t.advance(ee.Number(-3), 'days'), t)
        gpm_3day = gpm_3day.sum().divide(gpm_3day.count()).rename('prec_3')

        gpm_7day = gpm.filterDate(t.advance(ee.Number(-7), 'days'), t)
        gpm_7day = gpm_7day.sum().divide(gpm_7day.count()).rename('prec_7')

        gpm_30day = gpm.sum().divide(gpm.count()).rename('prec_30')

        return image.addBands(gpm_closest).addBands(gpm_3day).addBands(gpm_7day).addBands(gpm_30day)

    def addSRTM(image):

        srtm = ee.Image('USGS/SRTMGL1_003').resample()
        aspect = ee.Terrain.aspect(srtm).rename('aspect')
        slope = ee.Terrain.slope(srtm).rename('slope')

        return image.addBands(srtm.select('elevation').addBands(aspect).addBands(slope))
       
    def addGlobCover(image):
        
        lc = ee.Image("ESA/GLOBCOVER_L4_200901_200912_V2_3")
        return image.addBands(lc)
    
    image = addGPM(image)
    image = addGLDAS(image)
    image = addSRTM(image)
    image = addGlobCover(image)
                                    
    return image


def addTscans(image):
    
    def toLn(image):
        
        ln = image.select(['VV', 'VH']).log().rename(['VV', 'VH'])
        return image.addBands(ln, None, True)
    
    def toLin(image):
        
        lin = ee.Image(10).pow(image.select(['VV', 'VH']).divide(10)).rename(['VV', 'VH'])
        return image.addBands(lin, None, True)
        
    track_nr = image.get('relativeOrbitNumber_start').getInfo()
    orbit = image.get('orbitProperties_pass').getInfo()
    
    tSeries = s1_collection.create(
        region=image.geometry(),
        orbits=[orbit],
        start_date='2014-01-01',
        end_date='2021-01-01',
        add_ratio=False,
        add_ND_ratio=False,
        speckle_filter='NONE',
        radiometric_correction='TERRAIN',
        slope_correction_dict={'model': 'surface', 'dem': 'USGS/SRTMGL1_003', 'buffer': 50},
        db=False,
        outlier_removal='AGGRESSIVE'
        ) \
        .filterMetadata('relativeOrbitNumber_start', 'equals', track_nr)
    
    # create combined reducer
    reducer = ee.Reducer.mean() \
        .combine(ee.Reducer.stdDev(), '', True) \
        .combine(ee.Reducer.percentile([5, 95]), '', True)
    
    # create log timescan (k variables)
    tScanLn = tSeries.map(toLn).select(['VV', 'VH']).reduce(reducer).select(
    ['VV_mean', 'VV_stdDev', 'VV_p5', 'VV_p95', 'VH_mean', 'VH_stdDev', 'VH_p5', 'VH_p95'],
    ['kVV_mean', 'kVV_stdDev', 'kVV_p5', 'kVV_p95', 'kVH_mean', 'kVH_stdDev', 'kVH_p5', 'kVH_p95'])
    
    # create linear timescan
    tScanLin = tSeries.map(toLin).select(['VV', 'VH']).reduce(reducer)
    
    return image.addBands(tScanLn).addBands(tScanLin)

In [3]:
def mask(image):
    
    # single image mask
    vv = image.select('VV')
    vh = image.select('VH')
    lia = image.select('LIA')

    maskvv = vv.gte(-30).bitwiseAnd(vv.lt(0))
    maskvh = vh.gte(-30).bitwiseAnd(vh.lt(0))
    masklia = lia.gte(20).bitwiseAnd(vh.lt(45))
    
    mask_image = maskvv \
        .bitwiseAnd(maskvh) \
        .bitwiseAnd(masklia) 
    
    # mask based on SD (to exclude less variable pixels)
    kvvSd = image.select('kVV_stdDev').gt(0.1)
    kvhSd = image.select('kVH_stdDev').gt(0.1)
    
    # Globcover mask
    glbcvr = image.select('landcover')
    valLClist = [11, 14, 20, 30, 120, 140, 150]
    lcmask = glbcvr.eq(valLClist[0]) \
        .bitwiseOr(glbcvr.eq(valLClist[1])) \
        .bitwiseOr(glbcvr.eq(valLClist[2])) \
        .bitwiseOr(glbcvr.eq(valLClist[3])) \
        .bitwiseOr(glbcvr.eq(valLClist[4])) \
        .bitwiseOr(glbcvr.eq(valLClist[5])) \
        .bitwiseOr(glbcvr.eq(valLClist[6]))

    
    return image \
        .updateMask(mask_image \
            .bitwiseAnd(kvvSd) \
            .bitwiseAnd(kvhSd) #\
            #.bitwiseAnd(lcmask)
        )

def classify(i, image):
    
    geom = image.geometry().bounds(10)
    
    # add aux data
    image = addAux(image)
    image = addTscans(image)
    
    # apply mask
    image = mask(image)
    
    # get properties and construct out filename
    props = image.getInfo()['properties']
    scene_id = props['system:index']
    acq_date = scene_id[17:25]
    
    orbit = props['orbitProperties_pass']
    orbitDir = 'A' if orbit == 'ASCENDING' else 'D'
    track_nr = image.get('relativeOrbitNumber_start').getInfo()
    
    fileName = f'{orbitDir}{track_nr}_{acq_date}.{scene_id}.GWL'
    
    # get data for train model
    table = ee.FeatureCollection('users/andreasvollrath/final_training_noLC_noSRTM')
    label = 'GWL_rata'
    bandlist = [
     'VV', 'VH', 'VVVH_ratio', 'angle', 'LIA', #'layover', 'shadow', 'no_data_mask', 
     'precipitat', 'prec_3', 'prec_7', 'prec_30', 
     'sm_1', 'sm_3', 'sm_7', 'sm_30', 
     #'elevation', 'aspect', 'slope', 'landcover',
     'kVV_mean', 'kVV_stdDev', 'kVV_p5', 'kVV_p95', 'kVH_mean', 'kVH_stdDev', 'kVH_p5', 'kVH_p95', 
     'VV_mean', 'VV_stdDev', 'VV_p5', 'VV_p95', 'VH_mean', 'VH_stdDev', 'VH_p5', 'VH_p95'
    ]
    
    # train classifier
    trained = ee.Classifier.smileRandomForest(250).setOutputMode('REGRESSION').train(table, label, bandlist)
    
    # classify
    classified = image.select(bandlist).classify(trained)
    
    # create export task
    task = ee.batch.Export.image.toDrive(
                    image=classified,
                    description=f'{fileName}',
                    folder='gwl_model_ts_nolc_noSRTM_sumatra',
                    fileNamePrefix=f'{fileName}',
                    region=geom.coordinates().getInfo(),
                    scale=100,
                    maxPixels=1e12
                )

    # launch export task
    task.start()
    print(task.status())

In [ ]:
from multiprocessing import Process

start = '2015-01-01'
end = '2020-10-08'

# Sumatra AOI
geom = ee.Geometry.Polygon(
    [[[102.96601783701715,-0.7198776661874096],[102.96601783701715,-1.6617412280294859],[104.2157126612359,-1.6617412280294859],[104.2157126612359,-0.7198776661874096]]]
)

images = s1_collection.create(
    region=geom, 
    start_date=start,
    end_date=end, 
    add_ND_ratio=False,
    speckle_filter='QUEGAN',
    radiometric_correction='TERRAIN',
    slope_correction_dict={'model': 'surface', 'dem': 'USGS/SRTMGL1_003', 'buffer': 50}, #'CGIAR/SRTM90_V4'
    db=True
)

images_list = images.toList(images.size())
images_size = images_list.size().getInfo()

for i in range(images_size):

        if i == 0:
            continue 
            
        image = ee.Image(images_list.get(i))
        image_valid = image.select('VV').neq(0)
        
        # check on overlap
        image_geom = image_valid.reduceToVectors(
              geometry=geom,
              scale=100, 
              maxPixels=1e12
        )

        intersect = geom.intersection(image_geom, 10)
        inter_ratio = intersect.area(10).getInfo() / geom.area(10).getInfo()

        print(inter_ratio)
        
        if inter_ratio > 0.2:
            ec = None
            while ec is None != 0:
                p1 = Process(target=classify, args=(i, image, ), name='Process')
                p1.start()
                p1.join(timeout=240)
                p1.terminate()
                ec = p1.exitcode

0.7984538419818581
{'state': 'READY', 'description': 'D18_20150110.S1A_IW_GRDH_1SDV_20150110T224823_20150110T224848_004115_004FC5_04F2.GWL', 'creation_timestamp_ms': 1603723625829, 'update_timestamp_ms': 1603723625829, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'SOB6NKJE2WIJATF7EMFHAVJR', 'name': 'projects/earthengine-legacy/operations/SOB6NKJE2WIJATF7EMFHAVJR'}
0.45568145753772965
{'state': 'READY', 'description': 'A171_20150121.S1A_IW_GRDH_1SDV_20150121T112354_20150121T112423_004268_005312_34FF.GWL', 'creation_timestamp_ms': 1603724553329, 'update_timestamp_ms': 1603724553329, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'PUF4V3JD64JF3FK6E55Y7FCI', 'name': 'projects/earthengine-legacy/operations/PUF4V3JD64JF3FK6E55Y7FCI'}
0.5390674206357534
{'state': 'READY', 'description': 'A171_20150121.S1A_IW_GRDH_1SDV_20150121T112423_20150121T112448_004268_005312_ACFB.GWL', 'creation_timestamp_ms': 1603724804449, 'update_timestamp_ms': 1603724804449, 'start_timesta